<a href="https://colab.research.google.com/github/shivan222/census-data/blob/main/Census_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q sklearn

In [18]:
%tensorflow_version 2.x

In [19]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [34]:
dftrain = pd.read_csv('https://raw.githubusercontent.com/shivan222/census-data/main/census_train.csv')
dfeval = pd.read_csv('https://raw.githubusercontent.com/shivan222/census-data/main/census_eval.csv')
y_train = dftrain.pop('over50k')
y_eval = dfeval.pop('over50k')

In [54]:
CATEGORICAL_COLUMNS = ['workclass', 'education', 'marital_status', 'occupation', 'relationship',
                       'race', 'sex', 'native_country']
NUMERIC_COLUMNS = ['age', 'education_years']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [55]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [56]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpq260bsnd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [57]:
linear_est.train(train_input_fn)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpq260bsnd/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 260.077
INFO:tensorflow:loss = 0.28929695, step = 100 (0.391 sec)
INFO:tensorflow:global_step/sec: 598.689
INFO:tensorflow:loss = 0.18202743, step = 200 (0.163 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 220...
INFO:tensorflow:Saving checkpoints for 220 into /tmp/tmpq260bsnd/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving ch

In [58]:
result = linear_est.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-07-15T04:41:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpq260bsnd/model.ckpt-220
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.72070s
INFO:tensorflow:Finished evaluation at 2021-07-15-04:41:13
INFO:tensorflow:Saving dict for global step 220: accuracy = 0.80487806, accuracy_baseline = 0.7743902, auc = 0.81762075, auc_precision_recall = 0.50773424, average_loss = 0.455152, global_step = 220, label/mean = 0.22560975, loss = 0.47144058, precision = 0.6315789, prediction/mean = 0.14613605, recall = 0.3243243
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 220: /tmp/tmpq260bsnd/model.ckpt-220


In [59]:
clear_output()
print(result)

{'accuracy': 0.80487806, 'accuracy_baseline': 0.7743902, 'auc': 0.81762075, 'auc_precision_recall': 0.50773424, 'average_loss': 0.455152, 'label/mean': 0.22560975, 'loss': 0.47144058, 'precision': 0.6315789, 'prediction/mean': 0.14613605, 'recall': 0.3243243, 'global_step': 220}


In [90]:
result = list(linear_est.predict(eval_input_fn))

for i in range(50, 65):
  if y_eval[i] == 0:
    Over50k = "no"
  else:
    Over50k = "yes"

  print(dfeval.loc[i])
  print()
  print("Over $50k? " + Over50k)
  print("Probability of being Over $50k: ")
  print(result[i]['probabilities'][1])
  print("~~~~~~~~~~~~~~~")

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpq260bsnd/model.ckpt-220
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
age                                 44
workclass                      Private
education                      HS-grad
education_years                      9
marital_status      Married-civ-spouse
occupation                White-Collar
relationship                      Wife
race                             White
sex                             Female
hours_per_week                      40
native_country           United-States
Name: 50, dtype: object

Over $50k? no
Probability of being Over $50k: 
0.20733725
~~~~~~~~~~~~~~~
age                            20
workclass                 Private
education            Some-college
education_years                10
marital_status      Never-married
occupation            Blue-Collar
relationship        Not-in-family
r